In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Video Processing

In [ ]:
import cv2
import torch
from torchvision.transforms import functional as F
from PIL import Image
import math
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torch.load('./drone_detection_ssd_model.pth', map_location=torch.device('cpu'))
model.eval().to(device)

In [2]:
import cv2
import torch
from torchvision.transforms import functional as F
from PIL import Image
import math
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torch.load('/content/drive/MyDrive/drone_detection_ssd_model.pth', map_location=torch.device('cpu'))
model.eval().to(device)

def frame_to_tensor(frame):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img = F.to_tensor(img).to(device)
    return img

def draw_bounding_boxes(frame, boxes, labels):
    for box in boxes:
        x_min, y_min, x_max, y_max = map(int, box)
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    return frame

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    codec = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, codec, fps, (width, height))
    print(f"FPS: {fps}")

    idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img_tensor = frame_to_tensor(frame).unsqueeze(0)
        with torch.no_grad():
            outputs = model(img_tensor)

        predicted_box = torch.argmax(outputs[0]['scores'])
        labels = outputs[0]['labels'][predicted_box].cpu().numpy()
        boxes = outputs[0]['boxes'][predicted_box].unsqueeze(0).cpu().numpy()

        x_min, y_min, x_max, y_max = boxes[0]

        center_box = ((x_min + x_max) / 2, (y_min + y_max) / 2)
        delta_x = center_box[0] - width / 2
        delta_y = center_box[1] - height / 2
        angle_rad = math.atan2(delta_y, delta_x)
        angle_deg = math.degrees(angle_rad)
        angle_deg_from_y = (angle_deg + 90) % 360
        if angle_deg_from_y < 0:
          angle_deg_from_y += 360
        print(f"{idx} Angle in degrees from positive y-axis: {angle_deg_from_y}")
        idx += 1

        start_point = (int(width / 2), int(height / 2))
        end_point = (int(center_box[0]), int(center_box[1]))
        color = (255, 0, 0)
        thickness = 2
        image = cv2.line(frame, start_point, end_point, color, thickness)

        frame = draw_bounding_boxes(frame, boxes, labels)
        out.write(frame)

    cap.release()
    out.release()
    print("Processed video saved at:", output_path)

In [3]:
input_video_path = '/content/drive/MyDrive/drone_short_vid.mp4'
output_video_path = '/content/drive/MyDrive/output_video_with_bboxes.mp4'
process_video(input_video_path, output_video_path)

FPS: 25
0 Angle in degrees from positive y-axis: 187.128153076118
1 Angle in degrees from positive y-axis: 187.88848084408858
2 Angle in degrees from positive y-axis: 192.21612659410053
3 Angle in degrees from positive y-axis: 190.7327317778578
4 Angle in degrees from positive y-axis: 192.36407246369413
5 Angle in degrees from positive y-axis: 186.7017322476362
6 Angle in degrees from positive y-axis: 184.93080816206782
7 Angle in degrees from positive y-axis: 185.2655556679633
8 Angle in degrees from positive y-axis: 187.3468950656763
9 Angle in degrees from positive y-axis: 186.96166907302225
10 Angle in degrees from positive y-axis: 190.29703708490706
11 Angle in degrees from positive y-axis: 191.78248638472746
12 Angle in degrees from positive y-axis: 193.07161826032865
13 Angle in degrees from positive y-axis: 190.7629065486301
14 Angle in degrees from positive y-axis: 191.13923292102248
15 Angle in degrees from positive y-axis: 186.53257279093384
16 Angle in degrees from positive